# TreeModelBase
* Uses LightGBM to fit a tree model

In [ ]:
using LightGBM

import Flux: sigmoid
import SparseArrays: sparse
import Statistics: mean
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb");
@nbinclude("EnsembleInputs.ipynb")
@nbinclude("ItemMetadata.ipynb")
@nbinclude("SuppressImplicit.ipynb");

## Training

In [ ]:
function augment_dataset(ds, y, w)
    LightGBM.LGBM_DatasetSetField(ds, "label", y)
    LightGBM.LGBM_DatasetSetField(ds, "weight", w)
    ds
end

function create_train_dataset(X, y, w, estimator)
    augment_dataset(
        LightGBM.LGBM_DatasetCreateFromMat(X, LightGBM.stringifyparams(estimator), false),
        y,
        w,
    )
end

function create_test_dataset(X, y, w, estimator, train_ds)
    augment_dataset(
        LightGBM.LGBM_DatasetCreateFromMat(
            X,
            LightGBM.stringifyparams(estimator),
            train_ds,
            false,
        ),
        y,
        w,
    )
end;

In [ ]:
function get_features(alphas, split, implicit)
    @info "getting features for $split"
    base_features = reduce(hcat, [read_alpha(x, split, implicit).rating for x in alphas])
    metadata_features = get_item_metadata_features(split, implicit)
    hcat(base_features, metadata_features)
end

function get_data(split::String, feature_alphas, target_alphas, implicit, error_model)
    X = get_features(feature_alphas, split, implicit)
    if implicit
        if error_model
            # cross entropy loss between the distribution and the true rating
            y =
                get_split(split, implicit).rating .*
                get_weights(split, implicit, "inverse") .* 0.5 -
                read_alpha(target_alphas, split, implicit).rating
            w = get_weights(split, implicit, "inverse")
        else
            @assert false
        end
    else
        y =
            get_split(split, implicit).rating -
            read_alpha(target_alphas, split, implicit).rating
        w = get_weights(split, implicit, "inverse")
    end
    if error_model
        y = abs.(y)
    end
    training_mask = get_split(split, implicit).user .<= num_users() * 0.9
    X_train, X_test = X[training_mask, :], X[.!training_mask, :]
    y_train, y_test = y[training_mask], y[.!training_mask]
    w_train, w_test = w[training_mask], w[.!training_mask]
    X_train, X_test, y_train, y_test, w_train, w_test
end

function get_data(
    splits::Vector{String},
    feature_alphas,
    target_alphas,
    implicit,
    error_model,
    estimator,
)
    data = []
    for split in splits
        push!(data, get_data(split, feature_alphas, target_alphas, implicit, error_model))
    end
    X_train = reduce(vcat, data[n][1] for n = 1:length(data))
    X_test = reduce(vcat, data[n][2] for n = 1:length(data))
    y_train = reduce(vcat, data[n][3] for n = 1:length(data))
    y_test = reduce(vcat, data[n][4] for n = 1:length(data))
    w_train = reduce(vcat, data[n][5] for n = 1:length(data))
    w_test = reduce(vcat, data[n][6] for n = 1:length(data))
    train_ds = create_train_dataset(X_train, y_train, w_train, estimator)
    test_ds = create_test_dataset(X_test, y_test, w_test, estimator, train_ds)
    train_ds, test_ds
end;


In [ ]:
function memory_efficient_predict!(estimator, features, preds)
    batch_size = 1024
    n = size(features)[1]
    @showprogress for iter = 1:Int(ceil(n / batch_size))
        range = (iter-1)*batch_size+1:min(iter * batch_size, n)
        batch = features[range, :]
        preds[range] .= convert.(Float32, predict(estimator, batch))
    end
end;

In [ ]:
function create_estimator(λ)
    LGBMRegression(
        num_iterations = 1000,
        early_stopping_round = 10,
        learning_rate = 0.1 * exp(λ[1]),
        feature_fraction = sigmoid(λ[2]),
        bagging_fraction = sigmoid(λ[3]),
        bagging_freq = 1,
        num_leaves = Int(round(1000 * exp(λ[4]))),
        min_data_in_leaf = Int(round(100 * exp(λ[5]))),
    )
end;

In [ ]:
function sparse_predict(estimator, split, implicit, alphas)
    splits = get_split(split, implicit)
    split_features = get_features(alphas, split, implicit)
    memory_efficient_predict!(estimator, split_features, splits.rating)
    sparse_preds = sparse(splits.user, splits.item, splits.rating, num_users(), num_items())    
end;

In [ ]:
function train_model(
    feature_alphas,
    target_alphas,
    implicit,
    training_splits::Vector{String},
    outdir,
    error_model;
    λ = nothing
)
    set_logging_outdir(outdir)

    # create lightgbm tree model
    if isnothing(λ)
        estimator = LGBMRegression(
        num_iterations = 1000,
        learning_rate = 0.1,
        early_stopping_round = 10,
        feature_fraction = 0.8,
        bagging_fraction = 0.9,
        bagging_freq = 1,
        num_leaves = 1000,
        min_data_in_leaf = 100,
    )
    else
        estimator = create_estimator(λ)
    end


    # get training data
    train_ds, test_ds = get_data(
        training_splits,
        feature_alphas,
        target_alphas,
        implicit,
        error_model,
        estimator,
    )

    # train model
    fit!(estimator, train_ds, test_ds)

    # save model
    @info "Saving model..."
    write_params(Dict("model" => estimator, "alphas" => feature_alphas), outdir)
    sparse_preds = [sparse_predict(estimator, split, implicit, feature_alphas) for split in all_nontraining_raw_splits]
    @info "concatenating sparse preds"
    sparse_preds = reduce(+, sparse_preds)
    @info "Average model value: $(mean(sparse_preds.nzval))"
    @info "Average model absolute value: $(mean(abs.(sparse_preds.nzval)))"
    write_alpha(sparse_preds, target_alphas, implicit, outdir; log_splits = false)
end;

In [ ]:
# we only need to tune once every now and then
# function nlopt_loss(λ)
#     implicit = false
#     linear_alphas = ["LinearExplicit", "LinearImplicit"]
#     all_features = [
#         explicit_raw_alphas
#         implicit_raw_alphas
#         nondirectional_raw_alphas
#         linear_alphas
#     ]
#     estimator = create_estimator(λ)
#     train_ds, test_ds = get_data(
#         ["validation"],
#         all_features,
#         ["LinearExplicit"],
#         implicit,
#         false,
#         estimator,
#     )
#     results = fit!(estimator, train_ds, test_ds, verbosity = -1)
#     loss = results["metrics"]["test_1"]["l2"][end]
#     @info loss, λ
#     loss
# end

# function nlopt_loss_no_lr(λ, grad)
#     # don't the learning rate
#     nlopt_loss([[0.0]; λ])
# end

# import NLopt
# opt = NLopt.Opt(:LN_NELDERMEAD, 4)
# opt.initial_step = 1
# opt.maxeval = 25
# opt.min_objective = nlopt_loss_no_lr
# minf, λ, ret = NLopt.optimize(opt, zeros(4))
# numevals = opt.numevals;